# **Pre-processing**

---

## **Import Libraries & Data Loading**

In [72]:
import os.path as osp

import pandas as pd

data_folder = osp.join('..', 'resource', 'dataset')

In [ ]:
city_attri_df = pd.read_csv(osp.join(data_folder, 'raw', 'city_attributes.csv'))

humidity_df = pd.read_csv(osp.join(data_folder, 'raw', 'humidity.csv'))
temp_df = pd.read_csv(osp.join(data_folder, 'raw','temperature.csv'))
pressure_df = pd.read_csv(osp.join(data_folder, 'raw', 'pressure.csv'))
wind_direct_df = pd.read_csv(osp.join(data_folder, 'raw', 'wind_direction.csv'))
wind_speed_df = pd.read_csv(osp.join(data_folder, 'raw', 'wind_speed.csv'))
weather_descript_df = pd.read_csv(osp.join(data_folder, 'raw', 'weather_description.csv'))

In [ ]:
def preprocess_features(
    feature_df: pd.DataFrame
) -> pd.DataFrame:
    # date & time
    feature_df = feature_df.copy(True)
    feature_df['datetime'] = pd.to_datetime(feature_df['datetime'])
    feature_df['date'] = feature_df['datetime'].dt.date

    feature_df.drop(columns=['datetime'], inplace=True)

    # fill missing values with forward fill
    feature_df.drop(0, axis=0, inplace=True)
    feature_df.ffill(inplace=True)


    # aggregate by date and take mean
    return (feature_df
        .groupby(feature_df['date'])
        .mean(numeric_only=True)
        .reset_index()
    )

## **Humidity**

In [75]:
humidity_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45253 entries, 0 to 45252
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   datetime           45253 non-null  object 
 1   Vancouver          43427 non-null  float64
 2   Portland           44804 non-null  float64
 3   San Francisco      44311 non-null  float64
 4   Seattle            44964 non-null  float64
 5   Los Angeles        45101 non-null  float64
 6   San Diego          44909 non-null  float64
 7   Las Vegas          44411 non-null  float64
 8   Phoenix            43945 non-null  float64
 9   Albuquerque        44543 non-null  float64
 10  Denver             43445 non-null  float64
 11  San Antonio        44689 non-null  float64
 12  Dallas             44934 non-null  float64
 13  Houston            45132 non-null  float64
 14  Kansas City        44741 non-null  float64
 15  Minneapolis        44743 non-null  float64
 16  Saint Louis        439

In [ ]:
humidity_df = preprocess_features(humidity_df)
humidity_df

,date,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
0,2012-10-01,78.272727,78.727273,83.000000,78.000000,88.000000,79.909091,20.727273,24.181818,48.909091,...,68.272727,56.272727,75.727273,68.000000,54.818182,62.909091,23.090909,51.000,51.00,50.000000
1,2012-10-02,77.375000,65.833333,65.958333,65.166667,64.916667,65.875000,16.166667,23.500000,39.625000,...,54.500000,50.500000,47.000000,68.750000,72.916667,69.708333,33.833333,53.500,53.50,52.750000
2,2012-10-03,71.625000,66.208333,47.083333,58.416667,44.875000,59.750000,15.458333,18.000000,33.458333,...,78.791667,68.541667,91.875000,86.041667,75.541667,77.208333,47.291667,69.375,69.75,62.291667
3,2012-10-04,49.541667,51.166667,60.166667,48.125000,66.625000,75.583333,15.166667,19.833333,21.625000,...,89.041667,95.000000,95.458333,84.750000,67.458333,64.625000,49.333333,71.250,71.25,63.375000
4,2012-10-05,43.125000,39.750000,69.666667,48.250000,70.250000,72.791667,14.166667,27.333333,17.958333,...,84.750000,93.041667,89.833333,73.333333,65.666667,63.416667,46.875000,70.500,70.50,67.250000


In [78]:
humidity_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1887 entries, 0 to 1886
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               1887 non-null   object 
 1   Vancouver          1887 non-null   float64
 2   Portland           1887 non-null   float64
 3   San Francisco      1887 non-null   float64
 4   Seattle            1887 non-null   float64
 5   Los Angeles        1887 non-null   float64
 6   San Diego          1887 non-null   float64
 7   Las Vegas          1887 non-null   float64
 8   Phoenix            1887 non-null   float64
 9   Albuquerque        1887 non-null   float64
 10  Denver             1887 non-null   float64
 11  San Antonio        1887 non-null   float64
 12  Dallas             1887 non-null   float64
 13  Houston            1887 non-null   float64
 14  Kansas City        1887 non-null   float64
 15  Minneapolis        1887 non-null   float64
 16  Saint Louis        1887 

## **Pressure**

In [79]:
pressure_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45253 entries, 0 to 45252
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   datetime           45253 non-null  object 
 1   Vancouver          41019 non-null  float64
 2   Portland           45249 non-null  float64
 3   San Francisco      44438 non-null  float64
 4   Seattle            45240 non-null  float64
 5   Los Angeles        45001 non-null  float64
 6   San Diego          45078 non-null  float64
 7   Las Vegas          45165 non-null  float64
 8   Phoenix            44659 non-null  float64
 9   Albuquerque        44797 non-null  float64
 10  Denver             44710 non-null  float64
 11  San Antonio        45236 non-null  float64
 12  Dallas             45193 non-null  float64
 13  Houston            45244 non-null  float64
 14  Kansas City        45132 non-null  float64
 15  Minneapolis        45236 non-null  float64
 16  Saint Louis        451

In [ ]:
pressure_df = preprocess_features(pressure_df)
pressure_df

,date,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
0,2012-10-01,NaN,1024.000000,1009.727273,1030.000000,1013.000,1013.000000,1018.000000,1013.000000,1024.000000,...,1014.000000,1012.000000,875.545455,1014.000000,984.000000,1012.000000,1010.000000,1013.0,1013.0,990.000000
1,2012-10-02,972.800000,1023.583333,1011.250000,1034.291667,1012.750,1012.791667,1018.041667,1012.708333,1022.583333,...,1013.791667,1011.958333,890.625000,1013.791667,984.583333,1012.583333,1011.458333,1013.0,1013.0,990.500000
2,2012-10-03,1009.750000,1021.083333,1011.041667,1028.833333,1009.875,1010.666667,1013.833333,1011.041667,1021.833333,...,1014.583333,1015.541667,1013.083333,1013.416667,986.000000,1013.750000,1012.958333,1013.0,1013.0,990.791667
3,2012-10-04,1018.416667,1022.875000,1010.583333,1032.791667,1012.375,1013.083333,1011.583333,1011.791667,1018.375000,...,1016.458333,1016.875000,1018.000000,1018.208333,986.541667,1014.208333,1013.208333,1013.0,1013.0,991.000000
4,2012-10-05,1023.166667,1022.916667,1015.041667,1025.458333,1017.125,1017.875000,1013.625000,1015.166667,1021.000000,...,1019.625000,1019.250000,1019.458333,1021.375000,984.166667,1012.541667,1011.916667,1013.0,1013.0,989.750000


In [82]:
pressure_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1887 entries, 0 to 1886
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               1887 non-null   object 
 1   Vancouver          1886 non-null   float64
 2   Portland           1887 non-null   float64
 3   San Francisco      1887 non-null   float64
 4   Seattle            1887 non-null   float64
 5   Los Angeles        1887 non-null   float64
 6   San Diego          1887 non-null   float64
 7   Las Vegas          1887 non-null   float64
 8   Phoenix            1887 non-null   float64
 9   Albuquerque        1887 non-null   float64
 10  Denver             1887 non-null   float64
 11  San Antonio        1887 non-null   float64
 12  Dallas             1887 non-null   float64
 13  Houston            1887 non-null   float64
 14  Kansas City        1887 non-null   float64
 15  Minneapolis        1887 non-null   float64
 16  Saint Louis        1887 

## **Temperature**

In [83]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45253 entries, 0 to 45252
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   datetime           45253 non-null  object 
 1   Vancouver          44458 non-null  float64
 2   Portland           45252 non-null  float64
 3   San Francisco      44460 non-null  float64
 4   Seattle            45250 non-null  float64
 5   Los Angeles        45250 non-null  float64
 6   San Diego          45252 non-null  float64
 7   Las Vegas          45252 non-null  float64
 8   Phoenix            45250 non-null  float64
 9   Albuquerque        45252 non-null  float64
 10  Denver             45252 non-null  float64
 11  San Antonio        45252 non-null  float64
 12  Dallas             45249 non-null  float64
 13  Houston            45250 non-null  float64
 14  Kansas City        45252 non-null  float64
 15  Minneapolis        45240 non-null  float64
 16  Saint Louis        452

In [ ]:
temp_df = preprocess_features(temp_df)
temp_df

,date,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
0,2012-10-01,284.620769,282.118197,289.416642,281.767262,291.846501,291.573495,293.358911,296.701739,285.476208,...,286.043165,288.569420,285.887980,287.371091,306.621486,304.248983,310.158846,304.400000,304.400000,303.500000
1,2012-10-02,286.145190,286.137728,292.958306,285.156888,295.890450,295.291472,297.248385,301.211968,289.771821,...,289.239595,290.892389,286.937931,289.013090,302.226773,302.787467,306.759071,303.900000,303.900000,302.675000
2,2012-10-03,285.528125,289.599792,296.929167,287.673958,299.008542,297.878750,300.691875,302.867083,291.205417,...,290.353542,290.065625,287.374583,289.020833,301.194375,301.687917,303.289583,301.561042,301.502500,301.258125
3,2012-10-04,284.373333,286.482500,295.687083,284.391667,295.997917,296.080833,301.820000,302.232917,293.096250,...,293.633750,291.987083,286.860833,290.043750,300.094167,299.940000,301.770208,299.139167,299.139167,298.924167
4,2012-10-05,283.757292,288.286042,290.635417,284.756250,292.948333,293.894375,300.628542,301.811250,292.829167,...,294.015833,294.043542,287.535208,289.517292,299.712083,300.153125,299.860000,298.877500,298.877500,297.547500


In [86]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1887 entries, 0 to 1886
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               1887 non-null   object 
 1   Vancouver          1887 non-null   float64
 2   Portland           1887 non-null   float64
 3   San Francisco      1887 non-null   float64
 4   Seattle            1887 non-null   float64
 5   Los Angeles        1887 non-null   float64
 6   San Diego          1887 non-null   float64
 7   Las Vegas          1887 non-null   float64
 8   Phoenix            1887 non-null   float64
 9   Albuquerque        1887 non-null   float64
 10  Denver             1887 non-null   float64
 11  San Antonio        1887 non-null   float64
 12  Dallas             1887 non-null   float64
 13  Houston            1887 non-null   float64
 14  Kansas City        1887 non-null   float64
 15  Minneapolis        1887 non-null   float64
 16  Saint Louis        1887 

## **Wind Direction**

In [87]:
wind_direct_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45253 entries, 0 to 45252
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   datetime           45253 non-null  object 
 1   Vancouver          44458 non-null  float64
 2   Portland           45252 non-null  float64
 3   San Francisco      44459 non-null  float64
 4   Seattle            45252 non-null  float64
 5   Los Angeles        45252 non-null  float64
 6   San Diego          45252 non-null  float64
 7   Las Vegas          45248 non-null  float64
 8   Phoenix            45252 non-null  float64
 9   Albuquerque        45252 non-null  float64
 10  Denver             45252 non-null  float64
 11  San Antonio        45252 non-null  float64
 12  Dallas             45252 non-null  float64
 13  Houston            45251 non-null  float64
 14  Kansas City        45252 non-null  float64
 15  Minneapolis        45252 non-null  float64
 16  Saint Louis        452

In [ ]:
wind_direct_df = preprocess_features(wind_direct_df)
wind_direct_df

,date,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
0,2012-10-01,61.818182,57.727273,122.363636,32.272727,0.000000,0.000000,65.727273,7.727273,360.000000,...,275.000000,261.363636,235.727273,61.363636,155.000000,248.181818,30.000000,336.000000,336.000000,329.000000
1,2012-10-02,209.500000,214.041667,102.500000,126.583333,49.833333,95.833333,168.375000,22.833333,303.458333,...,278.041667,268.125000,260.583333,70.416667,160.791667,136.166667,90.000000,298.750000,298.750000,275.500000
2,2012-10-03,162.916667,228.333333,111.250000,97.708333,63.375000,142.708333,157.916667,127.833333,132.708333,...,96.416667,118.916667,227.583333,154.958333,132.958333,207.500000,139.375000,261.500000,262.500000,245.291667
3,2012-10-04,87.166667,206.750000,162.500000,220.416667,19.458333,119.166667,218.958333,94.666667,144.541667,...,100.500000,71.958333,101.583333,188.750000,185.000000,229.291667,168.541667,267.208333,267.208333,286.166667
4,2012-10-05,147.916667,182.250000,205.833333,103.750000,119.750000,177.416667,214.375000,124.000000,225.875000,...,70.000000,49.583333,66.333333,245.833333,195.666667,200.291667,116.458333,270.250000,270.250000,288.500000


In [90]:
wind_direct_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1887 entries, 0 to 1886
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               1887 non-null   object 
 1   Vancouver          1887 non-null   float64
 2   Portland           1887 non-null   float64
 3   San Francisco      1887 non-null   float64
 4   Seattle            1887 non-null   float64
 5   Los Angeles        1887 non-null   float64
 6   San Diego          1887 non-null   float64
 7   Las Vegas          1887 non-null   float64
 8   Phoenix            1887 non-null   float64
 9   Albuquerque        1887 non-null   float64
 10  Denver             1887 non-null   float64
 11  San Antonio        1887 non-null   float64
 12  Dallas             1887 non-null   float64
 13  Houston            1887 non-null   float64
 14  Kansas City        1887 non-null   float64
 15  Minneapolis        1887 non-null   float64
 16  Saint Louis        1887 

## **Wind Speed**

In [91]:
wind_speed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45253 entries, 0 to 45252
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   datetime           45253 non-null  object 
 1   Vancouver          44458 non-null  float64
 2   Portland           45252 non-null  float64
 3   San Francisco      44459 non-null  float64
 4   Seattle            45252 non-null  float64
 5   Los Angeles        45252 non-null  float64
 6   San Diego          45252 non-null  float64
 7   Las Vegas          45238 non-null  float64
 8   Phoenix            45251 non-null  float64
 9   Albuquerque        45249 non-null  float64
 10  Denver             45251 non-null  float64
 11  San Antonio        45252 non-null  float64
 12  Dallas             45251 non-null  float64
 13  Houston            45250 non-null  float64
 14  Kansas City        45252 non-null  float64
 15  Minneapolis        45250 non-null  float64
 16  Saint Louis        452

In [ ]:
wind_speed_df = preprocess_features(wind_speed_df)
wind_speed_df

,date,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
0,2012-10-01,0.000000,0.000000,1.636364,0.000000,0.000000,0.000000,0.000000,1.636364,4.000000,...,3.181818,6.363636,3.272727,3.000000,2.363636,0.454545,8.000000,2.000000,2.00,2.000000
1,2012-10-02,0.000000,1.291667,1.708333,0.625000,0.375000,1.125000,1.375000,1.000000,3.166667,...,2.333333,4.833333,1.666667,3.250000,2.166667,2.416667,7.541667,3.000000,3.00,2.000000
2,2012-10-03,0.541667,2.625000,2.375000,1.583333,0.541667,1.083333,1.250000,1.250000,2.958333,...,1.833333,2.416667,1.333333,4.083333,1.083333,2.500000,5.125000,4.708333,4.75,2.166667
3,2012-10-04,0.625000,4.625000,2.791667,3.375000,0.250000,1.000000,1.958333,0.916667,1.916667,...,1.833333,1.875000,1.000000,2.833333,1.041667,2.041667,5.583333,3.250000,3.25,2.791667
4,2012-10-05,0.416667,3.708333,2.125000,2.625000,0.916667,1.708333,3.083333,1.291667,2.583333,...,0.416667,1.375000,1.708333,1.916667,1.458333,2.166667,6.041667,2.250000,2.25,2.000000


In [94]:
wind_speed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1887 entries, 0 to 1886
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               1887 non-null   object 
 1   Vancouver          1887 non-null   float64
 2   Portland           1887 non-null   float64
 3   San Francisco      1887 non-null   float64
 4   Seattle            1887 non-null   float64
 5   Los Angeles        1887 non-null   float64
 6   San Diego          1887 non-null   float64
 7   Las Vegas          1887 non-null   float64
 8   Phoenix            1887 non-null   float64
 9   Albuquerque        1887 non-null   float64
 10  Denver             1887 non-null   float64
 11  San Antonio        1887 non-null   float64
 12  Dallas             1887 non-null   float64
 13  Houston            1887 non-null   float64
 14  Kansas City        1887 non-null   float64
 15  Minneapolis        1887 non-null   float64
 16  Saint Louis        1887 

## **Save preprocessed data**

In [40]:
humidity_df.to_csv(osp.join(data_folder, 'preprocessed', 'humidity.csv'), index=False)
pressure_df.to_csv(osp.join(data_folder, 'preprocessed', 'pressure.csv'), index=False)
temp_df.to_csv(osp.join(data_folder, 'preprocessed', 'temperature.csv'), index=False)
wind_direct_df.to_csv(osp.join(data_folder, 'preprocessed', 'wind_direction.csv'), index=False)
wind_speed_df.to_csv(osp.join(data_folder, 'preprocessed', 'wind_speed.csv'), index=False)